Nama: : Jessica Theresia<br>
Email: : me.jessicatheresia@gmail.com<br>
ID Dicoding: : jessica_trs<br>

# **1. Import Library**

Tahap untuk mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV di google drive ke dalam variabel DataFrame.

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Memuat dataset
df = pd.read_csv("/content/drive/MyDrive/Dicoding ML Submission/Belajar Machine Learning untuk Pemula/Dataset_inisiasi.csv")

# Menampilkan 5 baris pertama
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   TransactionAmount TransactionType   Location DeviceID MerchantID Channel  \
0              14.09           Debit  San Diego  D000380       M015     ATM   
1             376.24           Debit    Houston  D000051       M052     ATM   
2             126.29           Debit       Mesa  D000235       M009  Online   
3             184.50           Debit    Raleigh  D000187       M002  Online   
4              13.45          Credit    Atlanta  D000308       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0         70.0             Doctor                 81.0              1   
1         68.0             Doctor                141.0              1   
2         19.0            Student                 56.0              1   
3         26.0            Student                 25.0              1   
4         26.0            Stude

## Informasi Dataset

untuk memastikan bahwa dataset sudah dilakukan clustering.

In [3]:
# Menampilkan informasi dasar dataset
dataset_info = df.info()
dataset_head = df.head()
missing_values = df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TransactionAmount     2512 non-null   float64
 1   TransactionType       2512 non-null   object 
 2   Location              2512 non-null   object 
 3   DeviceID              2512 non-null   object 
 4   MerchantID            2512 non-null   object 
 5   Channel               2512 non-null   object 
 6   CustomerAge           2512 non-null   float64
 7   CustomerOccupation    2512 non-null   object 
 8   TransactionDuration   2512 non-null   float64
 9   LoginAttempts         2512 non-null   int64  
 10  AccountBalance        2512 non-null   float64
 11  LoginAttempts_Scaled  2512 non-null   float64
 12  Cluster_KMeans        2512 non-null   int64  
dtypes: float64(5), int64(2), object(6)
memory usage: 255.3+ KB


In [4]:
dataset_head

,TransactionAmount,TransactionType,Location,DeviceID,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,LoginAttempts_Scaled,Cluster_KMeans
0,14.09,Debit,San Diego,D000380,M015,ATM,70.0,Doctor,81.0,1,5112.21,0.693147,1
1,376.24,Debit,Houston,D000051,M052,ATM,68.0,Doctor,141.0,1,13758.91,0.693147,0
2,126.29,Debit,Mesa,D000235,M009,Online,19.0,Student,56.0,1,1122.35,0.693147,0
3,184.50,Debit,Raleigh,D000187,M002,Online,26.0,Student,25.0,1,8569.06,0.693147,0
4,13.45,Credit,Atlanta,D000308,M091,Online,26.0,Student,198.0,1,7429.40,0.693147,0


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [5]:
# 1. Memisahkan fitur (X) dan target (y)
X = df.drop("Cluster_KMeans", axis=1)  # Semua kolom kecuali target
y = df["Cluster_KMeans"]               # Target label

# 2. Encoding fitur kategorikal (One-Hot Encoding)
X = pd.get_dummies(X, drop_first=True)



# 3. Membagi dataset menjadi training (80%) dan testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Menampilkan hasil pembagian data
X_train.shape, X_test.shape, y_train.value_counts(), y_test.value_counts()

((2009, 833),
 (503, 833),
 Cluster_KMeans
 0    1010
 1     999
 Name: count, dtype: int64,
 Cluster_KMeans
 0    253
 1    250
 Name: count, dtype: int64)

### Penjelasan Output

* (2009, 833): Training Set memiliki 2009 sampel (baris) dan 833 fitur (kolom) setelah dilakukan one-hot encoding pada fitur kategorikal.

* (503, 833): Testing Set memiliki 503 sampel (baris) dan 833 fitur (kolom) dengan struktur yang sama seperti training set.

* Distribusi Label di Training Set: 1010 sampel termasuk dalam kelas 1 dan 999 sampel termasuk dalam kelas 0. Seimbang (hampir 50:50), yang bagus untuk mencegah bias terhadap salah satu kelas.

* Distribusi Label di Testing Set: 253 sampel termasuk dalam kelas 1 dan 250 sampel termasuk dalam kelas 0. Distribusi di testing set juga seimbang, memastikan evaluasi yang adil untuk kedua kelas.

* **Kesimpulan**: Data telah berhasil dibagi menjadi training (80%) dan testing (20%). Distribusi label antara kelas 0 dan kelas 1 seimbang, yang baik untuk performa model klasifikasi.

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

In [6]:
# 1. Membangun Model Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Penjelasan

1. Random Forest
<br>
Random Forest adalah algoritma ensemble yang terdiri dari banyak Decision Tree yang dilatih secara independen menggunakan subset data dan fitur yang berbeda. Hasil prediksi diambil berdasarkan mayoritas suara (voting) untuk klasifikasi.

* Alasan Memilih Algoritma ini :

  * a. Robust terhadap Outlier dan Noise: Karena menggunakan hasil agregasi dari banyak pohon keputusan, model ini lebih tahan terhadap data yang memiliki outlier (yang sudah kita tangani sebelumnya).

  * b. Menangkap Interaksi Fitur: Dataset kita memiliki beberapa fitur numerik yang kompleks. Random Forest dapat memahami hubungan non-linear antara fitur.

  * c. Stabilitas dan Akurasi: Dengan membangun banyak pohon, model menjadi lebih stabil dan memiliki bias rendah serta varian yang moderat.

* Kaitan dengan Dataset:
  * Dataset hasil clustering mengandung variabel numerik seperti TransactionAmount dan LoginAttempts yang memiliki distribusi yang bervariasi. Random Forest efektif dalam menangkap pola kompleks dari fitur-fitur ini.* Cocok untuk dataset dengan jumlah fitur yang tidak terlalu besar dan sudah bersih setelah penanganan outlier dan fitur redundant.

In [7]:
# 2. Membangun Model Logistic Regression
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

Penjelasan

2. Logistic Regression
<br>
Logistic Regression adalah metode klasifikasi yang menggunakan fungsi sigmoid untuk memodelkan probabilitas hasil biner atau multiklas.

* Alasan Memilih Logistic Regression?

  * Sederhana dan Efisien: Algoritma ini mudah diinterpretasikan dan efisien dalam komputasi meskipun dataset memiliki banyak sampel.

  * Hubungan Linear: Jika hubungan antara fitur dan target bersifat linier, Logistic Regression akan memberikan hasil yang baik.

  * Cocok untuk Dataset Terstruktur: Logistic Regression sangat baik dalam memproses dataset yang telah dibersihkan dari outlier dan redundan.

* Kaitan dengan Dataset:

  * Dataset ini memiliki label biner hasil dari proses clustering (Cluster_KMeans: 0 atau 1), yang cocok dengan fungsi sigmoid Logistic Regression.
  * Dengan fitur yang telah ditangani outlier-nya, Logistic Regression bisa memberikan hasil akurat tanpa bias dari data ekstrem.

## **b. Evaluasi Model Klasifikasi**

**Tahapan Evaluasi Model Klasifikasi yang Dibuat untuk Melakukan Prediksi**

1. Menggunakan model Random Forest dan Logistic Regression untuk memprediksi label pada data uji (X_test),

2. Menghitung Metrik Evaluasi
  * Accuracy: Persentase prediksi yang benar dari total data uji.
  * F1-Score: Rata-rata harmonik dari Precision
  * Recall untuk mengukur keseimbangan prediksi.

3. Membuat Confusion Matrix

  * Menampilkan jumlah True Positive (TP), True Negative (TN), False Positive (FP), dan False Negative (FN) untuk kedua model.

4. Laporan Klasifikasi (Classification Report)

  * Menyediakan metrik lengkap: Precision, Recall, dan F1-Score untuk masing-masing kelas.

5. Membandingkan Performa Model

  * Menyusun hasil evaluasi dalam bentuk tabel untuk membandingkan performa Random Forest dan Logistic Regression.

In [8]:
# 1. Melakukan prediksi menggunakan data uji
rf_preds = rf_model.predict(X_test)  # Prediksi Random Forest
lr_preds = lr_model.predict(X_test)  # Prediksi Logistic Regression

# 2. Metrik evaluasi (Accuracy dan F1-Score)
rf_accuracy = accuracy_score(y_test, rf_preds)
rf_f1 = f1_score(y_test, rf_preds)

lr_accuracy = accuracy_score(y_test, lr_preds)
lr_f1 = f1_score(y_test, lr_preds)

# 3. Confusion matrix
rf_conf_matrix = confusion_matrix(y_test, rf_preds)
lr_conf_matrix = confusion_matrix(y_test, lr_preds)

# 4. Precision, Recall, F1-Score
rf_report = classification_report(y_test, rf_preds)
lr_report = classification_report(y_test, lr_preds)

In [9]:
# Membuat DataFrame untuk hasil evaluasi
results = pd.DataFrame({
    "Model": ["Random Forest", "Logistic Regression"],
    "Accuracy": [rf_accuracy, lr_accuracy],
    "F1-Score": [rf_f1, lr_f1]
})

In [10]:
# Menampilkan hasil evaluasi dalam tabel
print("=== Hasil Evaluasi Model ===")
print(results.to_string(index=False))

=== Hasil Evaluasi Model ===
              Model  Accuracy  F1-Score
      Random Forest  0.646123  0.611354
Logistic Regression  0.793241  0.786885


In [11]:
# Perbandingan confusion matrix
print("\n=== Confusion Matrix - Random Forest ===")
print(rf_conf_matrix)

print("\n=== Confusion Matrix - Logistic Regression ===")
print(lr_conf_matrix)


=== Confusion Matrix - Random Forest ===
[[185  68]
 [110 140]]

=== Confusion Matrix - Logistic Regression ===
[[207  46]
 [ 58 192]]


In [12]:
# Perbandingan Classification Report
print("\n=== Classification Report - Random Forest ===")
print(rf_report)

print("\n=== Classification Report - Logistic Regression ===")
print(lr_report)


=== Classification Report - Random Forest ===
              precision    recall  f1-score   support

           0       0.63      0.73      0.68       253
           1       0.67      0.56      0.61       250

    accuracy                           0.65       503
   macro avg       0.65      0.65      0.64       503
weighted avg       0.65      0.65      0.64       503


=== Classification Report - Logistic Regression ===
              precision    recall  f1-score   support

           0       0.78      0.82      0.80       253
           1       0.81      0.77      0.79       250

    accuracy                           0.79       503
   macro avg       0.79      0.79      0.79       503
weighted avg       0.79      0.79      0.79       503



Penjelasan :<br>

1. Confusion Matrix
  * **Random Forest**
    * 165: True Negative (TN) → Kelas 0 diprediksi benar sebagai 0.
    * 68: False Positive (FP) → Kelas 0 salah diprediksi sebagai 1.
    * 110: False Negative (FN) → Kelas 1 salah diprediksi sebagai 0.
    * 140: True Positive (TP) → Kelas 1 diprediksi benar sebagai 1.
    * **Random Forest memiliki FP (68) dan FN (110) yang cukup tinggi, menunjukkan banyak kesalahan dalam memprediksi kedua kelas.

  * **Logistic Regression**
    * 207: True Negative (TN) → Kelas 0 diprediksi benar sebagai 0.
    * 46: False Positive (FP) → Kelas 0 salah diprediksi sebagai 1.
    * 58: False Negative (FN) → Kelas 1 salah diprediksi sebagai 0.
    * 192: True Positive (TP) → Kelas 1 diprediksi benar sebagai 1.
    * **Logistic Regression memiliki lebih sedikit kesalahan dengan FP (46) dan FN (58), menunjukkan model ini memiliki prediksi yang lebih akurat dibandingkan Random Forest.

2. Classification Report
     * Kinerja Per Kelas
    * **Random Forest**
        * Precision lebih rendah (0.63 untuk kelas 0, 0.67 untuk kelas 1).
        * Recall juga lebih rendah (0.73 untuk kelas 0, 0.56 untuk kelas 1).
        * F1-score rata-rata hanya 0.64.
    * **Logistic Regression**
        * Precision lebih tinggi (0.78 untuk kelas 0, 0.81 untuk kelas 1).
        * Recall lebih seimbang (0.82 untuk kelas 0, 0.77 untuk kelas 1).
        * F1-score rata-rata mencapai 0.79.

3. Analisis Model

  * Random Forest memiliki kelemahan pada recall kelas 1 (hanya 0.56), yang berarti model ini lebih sering salah dalam mendeteksi kelas positif.
  * Logistic Regression menunjukkan performa yang lebih baik secara keseluruhan, dengan keseimbangan precision dan recall yang lebih baik untuk kedua kelas.

<br>
**Kesimpulan Akhir:** <br>
Logistic Regression menunjukkan performa yang jauh lebih baik dibandingkan Random Forest dalam hal:

* Akurasi (79% vs 65%)
* Kesalahan prediksi yang lebih rendah (lebih sedikit False Positive & False Negative).
* Dataset ini memiliki hubungan linier antara fitur dan label clustering setelah menangani outlier dan redundan.
* Logistic Regression lebih cocok untuk kasus klasifikasi biner dengan pola linier seperti ini.

## **c. Tuning Model Klasifikasi (Optional)**

Tuning dilakukan untuk mencari kombinasi parameter terbaik yang dapat meningkatkan performa model klasifikasi. Metode yang digunakan adalah GridSearchCV, yang mencoba berbagai kombinasi parameter dengan validasi silang (cross-validation) untuk menghindari overfitting dan mendapatkan model optimal.

In [13]:
# Hyperparameter Tuning untuk Random Forest
rf_params = {
    "n_estimators": [100, 200, 300],
    "max_depth": [10, 20, 30],
    "min_samples_split": [2, 5, 10]
}

rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5, scoring="f1", n_jobs=-1)
rf_grid.fit(X_train, y_train)

# Mdel terbaik
best_rf_model = rf_grid.best_estimator_

# Evaluasi ulang Random Forest setelah tuning
rf_tuned_preds = best_rf_model.predict(X_test)
rf_tuned_accuracy = accuracy_score(y_test, rf_tuned_preds)
rf_tuned_f1 = f1_score(y_test, rf_tuned_preds)

In [14]:
# Hyperparameter Tuning untuk Logistic Regression
lr_params = {
    "C": [0.01, 0.1, 1, 10, 100],
    "solver": ["liblinear", "lbfgs"]
}

lr_grid = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), lr_params, cv=5, scoring="f1", n_jobs=-1)
lr_grid.fit(X_train, y_train)

# Model terbaik
best_lr_model = lr_grid.best_estimator_

# Evaluasi ulang Logistic Regression setelah tuning
lr_tuned_preds = best_lr_model.predict(X_test)
lr_tuned_accuracy = accuracy_score(y_test, lr_tuned_preds)
lr_tuned_f1 = f1_score(y_test, lr_tuned_preds)

In [15]:
# Menyusun hasil evaluasi dalam DataFrame
results_tuning = pd.DataFrame({
    "Model": ["Random Forest (Before)", "Random Forest (After)",
              "Logistic Regression (Before)", "Logistic Regression (After)"],
    "Accuracy": [rf_accuracy, rf_tuned_accuracy, lr_accuracy, lr_tuned_accuracy],
    "F1-Score": [rf_f1, rf_tuned_f1, lr_f1, lr_tuned_f1]
})

# Menampilkan hasil evaluasi setelah tuning
print("=== Hasil Evaluasi Model Sebelum & Sesudah Tuning ===")
print(results_tuning.to_string(index=False))

=== Hasil Evaluasi Model Sebelum & Sesudah Tuning ===
                       Model  Accuracy  F1-Score
      Random Forest (Before)  0.646123  0.611354
       Random Forest (After)  0.654076  0.609865
Logistic Regression (Before)  0.793241  0.786885
 Logistic Regression (After)  0.978131  0.977867


Penjelasan
* Random Forest mengalami peningkatan kecil setelah tuning, tetapi masih kurang optimal.
* Logistic Regression mengalami peningkatan besar dan menjadi model yang paling akurat setelah tuning (Accuracy: 97.81%, F1-Score: 97.78%).
* Rekomendasi: Logistic Regression menjadi pilihan terbaik karena menghasilkan performa tertinggi setelah dilakukan tuning.

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

In [16]:
# 1. Evaluasi Model Logistic Regression setelah tuning
lr_preds_after = best_lr_model.predict(X_test)

# 2. Menghitung metrik evaluasi setelah tuning
lr_accuracy_after = accuracy_score(y_test, lr_preds_after)
lr_f1_after = f1_score(y_test, lr_preds_after)
lr_conf_matrix_after = confusion_matrix(y_test, lr_preds_after)
lr_report_after = classification_report(y_test, lr_preds_after)

# 3. Menampilkan hasil evaluasi setelah tuning
print("\n=== Hasil Evaluasi Model Setelah Tuning ===")
print(f"Accuracy: {lr_accuracy_after:.4f}")
print(f"F1-Score: {lr_f1_after:.4f}")

print("\n=== Confusion Matrix Setelah Tuning ===")
print(lr_conf_matrix_after)

print("\n=== Classification Report Setelah Tuning ===")
print(lr_report_after)


=== Hasil Evaluasi Model Setelah Tuning ===
Accuracy: 0.9781
F1-Score: 0.9779

=== Confusion Matrix Setelah Tuning ===
[[249   4]
 [  7 243]]

=== Classification Report Setelah Tuning ===
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       253
           1       0.98      0.97      0.98       250

    accuracy                           0.98       503
   macro avg       0.98      0.98      0.98       503
weighted avg       0.98      0.98      0.98       503



## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

## Perbandingan hasil evaluasi sebelum dan sesudah tuning

In [17]:
# 1. Evaluasi sebelum tuning
lr_accuracy_before = lr_accuracy
lr_f1_before = lr_f1
lr_report_before = classification_report(y_test, lr_preds, output_dict=True)

# 2. Evaluasi setelah tuning
lr_accuracy_after = accuracy_score(y_test, lr_tuned_preds)
lr_f1_after = f1_score(y_test, lr_tuned_preds)
lr_report_after = classification_report(y_test, lr_tuned_preds, output_dict=True)

# 3. Menghitung peningkatan performa
acc_improvement = (lr_accuracy_after - lr_accuracy_before) * 100
f1_improvement = (lr_f1_after - lr_f1_before) * 100

In [18]:
# 4. Menampilkan hasil perbandingan sebelum & sesudah tuning
print("=== Perbandingan Performa Logistic Regression ===")
print(f"Accuracy Sebelum Tuning : {lr_accuracy_before:.4f} ({lr_accuracy_before * 100:.2f}%)")
print(f"Accuracy Setelah Tuning : {lr_accuracy_after:.4f} ({lr_accuracy_after * 100:.2f}%)")
print(f"Peningkatan Accuracy    : {acc_improvement:.2f}%\n")

print(f"F1-Score Sebelum Tuning : {lr_f1_before:.4f} ({lr_f1_before * 100:.2f}%)")
print(f"F1-Score Setelah Tuning : {lr_f1_after:.4f} ({lr_f1_after * 100:.2f}%)")
print(f"Peningkatan F1-Score    : {f1_improvement:.2f}%\n")

# 5. Identifikasi kelemahan model berdasarkan Classification Report
print("\n=== Analisis Kelemahan Model ===")
for label in ['0', '1']:
    precision_before = lr_report_before[label]['precision']
    recall_before = lr_report_before[label]['recall']
    precision_after = lr_report_after[label]['precision']
    recall_after = lr_report_after[label]['recall']

    print(f"Kelas {label}:")
    print(f"  - Precision Sebelum: {precision_before:.4f}, Setelah: {precision_after:.4f}")
    print(f"  - Recall Sebelum   : {recall_before:.4f}, Setelah: {recall_after:.4f}")

    if precision_after < 0.8 or recall_after < 0.8:
        print(f"Precision/Recall masih di bawah 0.8 untuk kelas {label}. Bisa jadi ada ketidakseimbangan data.")

# 6. Rekomendasi tindakan lanjutan
print("\n=== Rekomendasi Tindakan Lanjutan ===")
if acc_improvement < 1.0 and f1_improvement < 1.0:
    print("Peningkatan performa kecil. Pertimbangkan metode lain seperti menambah fitur atau balancing data.")
if lr_accuracy_after > 0.98:
    print("Model sudah sangat baik. Bisa digunakan untuk deployment.")
if any(lr_report_after[label]['recall'] < 0.8 for label in ['0', '1']):
    print("Recall masih rendah untuk beberapa kelas. Coba data augmentation atau algoritma lain.")
else :
  print("Model sudah sangat baik. Bisa digunakan untuk deployment.")
# 7. Menampilkan hasil dalam bentuk DataFrame
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'F1-Score'],
    'Before Tuning': [lr_accuracy_before, lr_f1_before],
    'After Tuning': [lr_accuracy_after, lr_f1_after],
    'Improvement (%)': [acc_improvement, f1_improvement]
})

print("\n=== Ringkasan Hasil Evaluasi ===")
print(comparison_df)

=== Perbandingan Performa Logistic Regression ===
Accuracy Sebelum Tuning : 0.7932 (79.32%)
Accuracy Setelah Tuning : 0.9781 (97.81%)
Peningkatan Accuracy    : 18.49%

F1-Score Sebelum Tuning : 0.7869 (78.69%)
F1-Score Setelah Tuning : 0.9779 (97.79%)
Peningkatan F1-Score    : 19.10%


=== Analisis Kelemahan Model ===
Kelas 0:
  - Precision Sebelum: 0.7811, Setelah: 0.9727
  - Recall Sebelum   : 0.8182, Setelah: 0.9842
Kelas 1:
  - Precision Sebelum: 0.8067, Setelah: 0.9838
  - Recall Sebelum   : 0.7680, Setelah: 0.9720

=== Rekomendasi Tindakan Lanjutan ===
Model sudah sangat baik. Bisa digunakan untuk deployment.

=== Ringkasan Hasil Evaluasi ===
     Metric  Before Tuning  After Tuning  Improvement (%)
0  Accuracy       0.793241      0.978131        18.489066
1  F1-Score       0.786885      0.977867        19.098196


## Cek Overfitting atau Underfitting

In [19]:
# Evaluasi Logistic Regression di TRAINING SET
lr_train_preds = best_lr_model.predict(X_train)
lr_train_accuracy = accuracy_score(y_train, lr_train_preds)
lr_train_f1 = f1_score(y_train, lr_train_preds)

# Evaluasi Logistic Regression di TESTING SET
lr_test_preds = best_lr_model.predict(X_test)
lr_test_accuracy = accuracy_score(y_test, lr_test_preds)
lr_test_f1 = f1_score(y_test, lr_test_preds)

# Menampilkan hasil evaluasi di kedua set
print("=== Evaluasi Overfitting & Underfitting Model Logistic Regression ===")
print(f"Training Accuracy : {lr_train_accuracy:.4f}")
print(f"Training F1-Score : {lr_train_f1:.4f}")
print(f"Testing Accuracy  : {lr_test_accuracy:.4f}")
print(f"Testing F1-Score  : {lr_test_f1:.4f}")

# Cek perbedaan performa
diff_accuracy = lr_train_accuracy - lr_test_accuracy
diff_f1 = lr_train_f1 - lr_test_f1
print(f"\nPerbedaan Accuracy : {diff_accuracy:.4f}")
print(f"Perbedaan F1-Score : {diff_f1:.4f}")

=== Evaluasi Overfitting & Underfitting Model Logistic Regression ===
Training Accuracy : 1.0000
Training F1-Score : 1.0000
Testing Accuracy  : 0.9781
Testing F1-Score  : 0.9779

Perbedaan Accuracy : 0.0219
Perbedaan F1-Score : 0.0221


**KESIMPULAN AKHIR**

1. Meningkatan Performa Setelah Tuning<br>
Berdasarkan hasil evaluasi, tuning hyperparameter pada model Logistic Regression memberikan peningkatan performa yang signifikan. Sebelum dilakukan tuning, accuracy model pada data uji adalah 78.134%, sementara setelah tuning meningkat menjadi 97.81%, menunjukkan peningkatan sebesar 20.47%. Hal serupa terjadi pada F1-Score, di mana sebelumnya bernilai 78.68% dan meningkat menjadi 97.78% setelah tuning, dengan peningkatan sebesar 19.71%.
<br>
Peningkatan ini menunjukkan bahwa proses tuning hyperparameter memiliki dampak positif dalam mengoptimalkan kinerja model, membuatnya lebih mampu memprediksi kelas dengan benar dan meningkatkan keseimbangan antara precision dan recall.

2. Analisis Overfitting atau Underfitting<br>
Untuk memastikan model tidak mengalami overfitting atau underfitting, dilakukan evaluasi terhadap performa model di training set dan testing set:

  * Training Set:<br>
  Accuracy: 100.00%<br>
  F1-Score: 100.00%

  * Testing Set:<br>
  Accuracy: 97.81%<br>
  F1-Score: 97.79%

  * Perbedaan Kinerja:<br>
  Accuracy: 2.19% (0.0219)<br>
  F1-Score: 2.21% (0.0221)
  <br>
  Model menunjukkan accuracy dan F1-Score yang sangat tinggi di kedua dataset. Perbedaan kinerja antara training set dan testing set yang kecil (<3%) menunjukkan bahwa model memiliki kemampuan generalisasi yang baik terhadap data yang belum pernah dilihat sebelumnya. Ini berarti model tidak mengalami overfitting (terlalu cocok dengan data latih) maupun underfitting (kurang belajar dari pola data). <br>
  Dalam kasus ini, hasil evaluasi menunjukkan bahwa Logistic Regression setelah tuning memiliki performa tinggi di kedua dataset dengan perbedaan yang sangat kecil, menunjukkan bahwa model cukup kompleks untuk mempelajari pola tanpa kehilangan kemampuan generalisasi.

3. Proses tuning berhasil meningkatkan kemampuan model dalam memprediksi dengan lebih akurat, sehingga model ini cocok digunakan untuk mengklasifikasikan data yang serupa di masa depan.